# upright 이게 아직까지는 굿인듯

In [2]:
'''
[Local] Upright Player
- Fix 1: SAE/Actor classes match the server code.
- Fix 2: 'reset()' starts at Bottom (Pi) to test swing-up recovery.
- Fix 3: 'TkAgg' backend force.
'''
import torch
import torch.nn as nn
import numpy as np
import math
import time
import matplotlib
try: matplotlib.use('TkAgg')
except: pass
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# --- Definitions (Must Match Server) ---
class DoublePendulumEnv:
    def __init__(self):
        self.gravity = 9.8; self.masscart = 1.0
        self.masspole1 = 0.5; self.masspole2 = 0.5
        self.length1 = 0.5; self.length2 = 0.5
        self.force_mag = 40.0; self.dt = 0.02
        self.friction_cart = 5e-4; self.friction_pole = 2e-3
        self.x_threshold = 4.0
        self.reset()

    def reset(self):
        # [TEST MODE] Start from Bottom to check recovery
        th1 = np.pi; th2 = np.pi 
        self.physical_state = np.array([0, 0, th1, 0, th2, 0], dtype=np.float32)
        return self.get_obs()

    def get_obs(self):
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        return np.array([x, x_dot, np.sin(th1), np.cos(th1), th1_dot, np.sin(th2), np.cos(th2), th2_dot], dtype=np.float32)

    def step(self, action):
        raw_action = np.clip(action, -1.0, 1.0)[0]
        force = np.sign(raw_action) * (raw_action ** 2) * self.force_mag
        
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        s1, c1, s2, c2 = math.sin(th1), math.cos(th1), math.sin(th2), math.cos(th2)
        s12, c12 = math.sin(th1 - th2), math.cos(th1 - th2)
        m1, m2, M, l1, l2, g = self.masspole1, self.masspole2, self.masscart, self.length1, self.length2, self.gravity

        term1 = (m1 + m2) * l1 * th1_dot**2 * s1
        term2 = m2 * l2 * th2_dot**2 * s2
        f_c, f_p1, f_p2 = -self.friction_cart*x_dot, -self.friction_pole*th1_dot, -self.friction_pole*th2_dot
        eff_force = force + f_c
        
        torque1 = (M + m1 + m2) * g * s1 - term1 * c1 - term2 * c12 + f_p1
        th1_acc = (torque1 - eff_force * c1) / ((4/3) * (M + m1 + m2) * l1)
        th2_acc = (g * s2 - th1_acc * l1 * c12 + f_p2/l2) / (l2 * (4/3))
        x_acc = (eff_force - term1 - term2) / (M + m1 + m2)

        x_dot += x_acc * self.dt; x += x_dot * self.dt
        th1_dot += th1_acc * self.dt; th1 += th1_dot * self.dt
        th2_dot += th2_acc * self.dt; th2 += th2_dot * self.dt
        
        if x < -self.x_threshold: x = -self.x_threshold; x_dot = 0
        elif x > self.x_threshold: x = self.x_threshold; x_dot = 0
            
        self.physical_state = np.array([x, x_dot, th1, th1_dot, th2, th2_dot], dtype=np.float32)
        return self.get_obs()

class SparseAutoEncoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 128), nn.ELU(), nn.Linear(128, 64), nn.ELU(), nn.Linear(64, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 64), nn.ELU(), nn.Linear(64, 128), nn.ELU(), nn.Linear(128, input_dim))
    def forward(self, x): return None, self.encoder(x)

class PPOActor(nn.Module):
    def __init__(self, latent_dim=32, action_dim=1):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(latent_dim, 128), nn.Tanh(), nn.Linear(128, 64), nn.Tanh())
        self.mu = nn.Linear(64, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
    def forward(self, z): return torch.tanh(self.mu(self.net(z))), None

def main():
    device = torch.device("cpu")
    print("Loading models (Upright Version)...")
    try:
        sae = SparseAutoEncoder(8, 32).to(device)
        actor = PPOActor(32, 1).to(device)
        sae.load_state_dict(torch.load("sae_upright.pth", map_location=device))
        actor.load_state_dict(torch.load("actor_upright.pth", map_location=device))
        sae.eval(); actor.eval()
        print("Loaded.")
    except: print("Error: Files not found."); return

    env = DoublePendulumEnv()
    obs = env.reset()
    
    plt.ion()
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_xlim(-5, 5); ax.set_ylim(-3, 3); ax.set_aspect('equal')
    ax.grid(True); ax.axvline(-4.0, c='k', ls='--'); ax.axvline(4.0, c='k', ls='--')
    cart, = ax.plot([], [], 'ks', ms=20)
    pole1, = ax.plot([], [], 'r-', lw=5); pole2, = ax.plot([], [], 'b-', lw=5)
    txt = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    print("Running... Close window to exit.")
    plt.show(block=False); plt.pause(0.1)
    
    while plt.fignum_exists(fig.number):
        s_t = torch.from_numpy(obs).float().unsqueeze(0).to(device)
        with torch.no_grad(): _, z = sae(s_t); mu, _ = actor(z)
        action = mu.item()
        
        obs = env.step([action])
        x, _, th1, _, th2, _ = env.physical_state
        
        p1x = x + 2*env.length1*math.sin(th1); p1y = 2*env.length1*math.cos(th1)
        p2x = p1x + 2*env.length2*math.sin(th2); p2y = p1y + 2*env.length2*math.cos(th2)
        
        cart.set_data([x], [0])
        pole1.set_data([x, p1x], [0, p1y])
        pole2.set_data([p1x, p2x], [p1y, p2y])
        txt.set_text(f"Action: {action:.2f}")
        fig.canvas.draw(); fig.canvas.flush_events(); plt.pause(0.01)

if __name__ == "__main__": main()

Loading models (Upright Version)...
Loaded.
Running... Close window to exit.


## advanced

In [ ]:
'''
[Local Player] Advanced Double Pendulum Viewer
- Loads: 'sae_advanced.pth', 'actor_advanced.pth'
- Features: Real-time rendering, Wall Collision visualization.
'''

import torch
import torch.nn as nn
import numpy as np
import math
import matplotlib
try: matplotlib.use('TkAgg')
except: pass
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# ==========================================
# 1. Definitions (Must match Server Code)
# ==========================================
class AdvancedRewardEnv:
    def __init__(self):
        self.gravity = 9.8; self.masscart = 1.0
        self.masspole1 = 0.5; self.masspole2 = 0.5
        self.length1 = 0.5; self.length2 = 0.5
        self.force_mag = 40.0; self.dt = 0.02 # Local visual speed
        self.x_threshold = 4.0
        self.friction_cart = 5e-4; self.friction_pole = 2e-3
        self.reset()

    def reset(self):
        # Start from Bottom to verify Swing-up capability
        th1 = np.pi; th2 = np.pi 
        self.physical_state = np.array([0, 0, th1, 0, th2, 0], dtype=np.float32)
        return self.get_obs()

    def get_obs(self):
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        return np.array([x, x_dot, np.sin(th1), np.cos(th1), th1_dot, np.sin(th2), np.cos(th2), th2_dot], dtype=np.float32)

    def step(self, action):
        raw_action = np.clip(action, -1.0, 1.0)[0]
        force = np.sign(raw_action) * (raw_action ** 2) * self.force_mag
        
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        s1, c1, s2, c2 = math.sin(th1), math.cos(th1), math.sin(th2), math.cos(th2)
        s12, c12 = math.sin(th1 - th2), math.cos(th1 - th2)
        m1, m2, M, l1, l2, g = self.masspole1, self.masspole2, self.masscart, self.length1, self.length2, self.gravity

        term1 = (m1 + m2) * l1 * th1_dot**2 * s1
        term2 = m2 * l2 * th2_dot**2 * s2
        f_c, f_p1, f_p2 = -self.friction_cart*x_dot, -self.friction_pole*th1_dot, -self.friction_pole*th2_dot
        eff_force = force + f_c
        
        torque1 = (M + m1 + m2) * g * s1 - term1 * c1 - term2 * c12 + f_p1
        th1_acc = (torque1 - eff_force * c1) / ((4/3) * (M + m1 + m2) * l1)
        th2_acc = (g * s2 - th1_acc * l1 * c12 + f_p2/l2) / (l2 * (4/3))
        x_acc = (eff_force - term1 - term2) / (M + m1 + m2)

        x_dot += x_acc * self.dt; x += x_dot * self.dt
        th1_dot += th1_acc * self.dt; th1 += th1_dot * self.dt
        th2_dot += th2_acc * self.dt; th2 += th2_dot * self.dt
        
        if x < -self.x_threshold: x = -self.x_threshold; x_dot = 0.0
        elif x > self.x_threshold: x = self.x_threshold; x_dot = 0.0
            
        self.physical_state = np.array([x, x_dot, th1, th1_dot, th2, th2_dot], dtype=np.float32)
        return self.get_obs()

class SparseAutoEncoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 128), nn.ELU(), nn.Linear(128, 64), nn.ELU(), nn.Linear(64, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 64), nn.ELU(), nn.Linear(64, 128), nn.ELU(), nn.Linear(128, input_dim))
    def forward(self, x): return None, self.encoder(x)

class PPOActor(nn.Module):
    def __init__(self, latent_dim=32, action_dim=1):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(latent_dim, 128), nn.Tanh(), nn.Linear(128, 64), nn.Tanh())
        self.mu = nn.Linear(64, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
    def forward(self, z): return torch.tanh(self.mu(self.net(z))), None

# ==========================================
# 2. Main Player
# ==========================================
def main():
    device = torch.device("cpu")
    print("Loading models...")
    try:
        sae = SparseAutoEncoder(8, 32).to(device)
        actor = PPOActor(32, 1).to(device)
        sae.load_state_dict(torch.load("sae_advanced.pth", map_location=device))
        actor.load_state_dict(torch.load("actor_advanced.pth", map_location=device))
        sae.eval(); actor.eval()
        print("Models loaded.")
    except Exception as e:
        print(f"Error: {e}\nEnsure 'sae_advanced.pth' and 'actor_advanced.pth' are present.")
        return

    env = AdvancedRewardEnv()
    obs = env.reset()
    
    plt.ion()
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_xlim(-5, 5); ax.set_ylim(-3, 3); ax.set_aspect('equal')
    ax.grid(True)
    ax.axvline(-4.0, c='k', ls='--'); ax.axvline(4.0, c='k', ls='--')
    
    cart, = ax.plot([], [], 'ks', ms=20)
    pole1, = ax.plot([], [], 'r-', lw=5, label='Inner')
    pole2, = ax.plot([], [], 'b-', lw=5, label='Outer')
    txt = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    ax.legend(loc='upper right')
    
    print("Running... Close window to exit.")
    plt.show(block=False); plt.pause(0.1)
    
    while plt.fignum_exists(fig.number):
        s_t = torch.from_numpy(obs).float().unsqueeze(0).to(device)
        with torch.no_grad(): _, z = sae(s_t); mu, _ = actor(z)
        action = mu.item()
        
        obs = env.step([action])
        x, _, th1, _, th2, _ = env.physical_state
        
        p1x = x + 2*env.length1*math.sin(th1); p1y = 2*env.length1*math.cos(th1)
        p2x = p1x + 2*env.length2*math.sin(th2); p2y = p1y + 2*env.length2*math.cos(th2)
        
        cart.set_data([x], [0])
        pole1.set_data([x, p1x], [0, p1y])
        pole2.set_data([p1x, p2x], [p1y, p2y])
        txt.set_text(f"Action: {action:.2f}")
        
        fig.canvas.draw(); fig.canvas.flush_events(); plt.pause(0.01)

    plt.ioff(); print("Closed.")

if __name__ == "__main__": main()

Loading models...
Models loaded.
Running... Close window to exit.


## easy

In [1]:
'''
[Local Player] Double Pendulum with Trajectory Trace
- Feature: Draws faint dots to trace the path of the outer pole's tip.
- Usage: Ensure 'sae_easy.pth' and 'actor_easy.pth' are in the same directory.
'''

import torch
import torch.nn as nn
import numpy as np
import math
import matplotlib
try: matplotlib.use('TkAgg')
except: pass
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# ==========================================
# 1. Definitions (Must match Server Code)
# ==========================================
class DoublePendulumEnv:
    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole1 = 0.5
        self.masspole2 = 0.5
        self.length1 = 0.5
        self.length2 = 0.5
        self.force_mag = 40.0
        self.dt = 0.02 # Local visual speed
        self.x_threshold = 4.0
        self.friction_cart = 5e-4
        self.friction_pole = 2e-3 
        self.reset()

    def reset(self):
        # Start from Bottom to verify Swing-up capability
        th1 = np.pi 
        th2 = np.pi 
        self.physical_state = np.array([0, 0, th1, 0, th2, 0], dtype=np.float32)
        return self.get_obs()

    def get_obs(self):
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        return np.array([x, x_dot, np.sin(th1), np.cos(th1), th1_dot, np.sin(th2), np.cos(th2), th2_dot], dtype=np.float32)

    def step(self, action):
        raw_action = np.clip(action, -1.0, 1.0)[0]
        force = np.sign(raw_action) * (raw_action ** 2) * self.force_mag
        
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        s1, c1 = math.sin(th1), math.cos(th1)
        s2, c2 = math.sin(th2), math.cos(th2)
        s12, c12 = math.sin(th1 - th2), math.cos(th1 - th2)
        m1, m2, M, l1, l2, g = self.masspole1, self.masspole2, self.masscart, self.length1, self.length2, self.gravity

        term1 = (m1 + m2) * l1 * th1_dot**2 * s1
        term2 = m2 * l2 * th2_dot**2 * s2
        f_c, f_p1, f_p2 = -self.friction_cart*x_dot, -self.friction_pole*th1_dot, -self.friction_pole*th2_dot
        eff_force = force + f_c
        
        torque1 = (M + m1 + m2) * g * s1 - term1 * c1 - term2 * c12 + f_p1
        th1_acc = (torque1 - eff_force * c1) / ((4/3) * (M + m1 + m2) * l1)
        th2_acc = (g * s2 - th1_acc * l1 * c12 + f_p2/l2) / (l2 * (4/3))
        x_acc = (eff_force - term1 - term2) / (M + m1 + m2)

        x_dot += x_acc * self.dt
        x += x_dot * self.dt
        th1_dot += th1_acc * self.dt
        th1 += th1_dot * self.dt
        th2_dot += th2_acc * self.dt
        th2 += th2_dot * self.dt
        
        if x < -self.x_threshold: x = -self.x_threshold; x_dot = 0.0
        elif x > self.x_threshold: x = self.x_threshold; x_dot = 0.0
            
        self.physical_state = np.array([x, x_dot, th1, th1_dot, th2, th2_dot], dtype=np.float32)
        return self.get_obs()

class SparseAutoEncoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 128), nn.ELU(), nn.Linear(128, 64), nn.ELU(), nn.Linear(64, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 64), nn.ELU(), nn.Linear(64, 128), nn.ELU(), nn.Linear(128, input_dim))
    def forward(self, x): return None, self.encoder(x)

class PPOActor(nn.Module):
    def __init__(self, latent_dim=32, action_dim=1):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(latent_dim, 128), nn.Tanh(), nn.Linear(128, 64), nn.Tanh())
        self.mu = nn.Linear(64, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
    def forward(self, z): return torch.tanh(self.mu(self.net(z))), None

# ==========================================
# 2. Main Player with Trace
# ==========================================
def main():
    device = torch.device("cpu")
    print("Loading models...")
    try:
        sae = SparseAutoEncoder(8, 32).to(device)
        actor = PPOActor(32, 1).to(device)
        
        # [주의] 저장된 파일명에 맞춰주세요 (예: sae_easy.pth)
        sae.load_state_dict(torch.load("sae_easy.pth", map_location=device))
        actor.load_state_dict(torch.load("actor_easy.pth", map_location=device))
        sae.eval(); actor.eval()
        print("Models loaded.")
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure 'sae_easy.pth' and 'actor_easy.pth' exist.")
        return

    env = DoublePendulumEnv()
    obs = env.reset()
    
    plt.ion()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(-5, 5); ax.set_ylim(-3, 3); ax.set_aspect('equal')
    ax.grid(True)
    ax.axvline(-4.0, c='k', ls='--'); ax.axvline(4.0, c='k', ls='--')
    
    # Trace Plot (궤적 표시용)
    trace_x, trace_y = [], []
    trace_line, = ax.plot([], [], 'o', color='gray', markersize=2, alpha=0.2, label='Tip Trace')
    
    cart, = ax.plot([], [], 'ks', ms=20, label='Cart')
    pole1, = ax.plot([], [], 'r-', lw=5, label='Inner')
    pole2, = ax.plot([], [], 'b-', lw=5, label='Outer')
    txt = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    ax.legend(loc='upper right')
    
    print("Running... Trace will appear as faint dots.")
    
    while plt.fignum_exists(fig.number):
        # Inference
        s_t = torch.from_numpy(obs).float().unsqueeze(0).to(device)
        with torch.no_grad(): _, z = sae(s_t); mu, _ = actor(z)
        action = mu.item()
        
        obs = env.step([action])
        x, _, th1, _, th2, _ = env.physical_state
        
        # Coordinates
        p1x = x + 2*env.length1*math.sin(th1)
        p1y = 2*env.length1*math.cos(th1)
        p2x = p1x + 2*env.length2*math.sin(th2)
        p2y = p1y + 2*env.length2*math.cos(th2)
        
        # [Trace Update]
        trace_x.append(p2x)
        trace_y.append(p2y)
        # 너무 길어지면 성능 저하 방지를 위해 최근 500개만 표시 (원하면 슬라이싱 제거 가능)
        if len(trace_x) > 1000: 
            trace_x.pop(0); trace_y.pop(0)
            
        trace_line.set_data(trace_x, trace_y)
        
        # Objects Update
        cart.set_data([x], [0])
        pole1.set_data([x, p1x], [0, p1y])
        pole2.set_data([p1x, p2x], [p1y, p2y])
        txt.set_text(f"Action: {action:.2f}")
        
        fig.canvas.draw(); fig.canvas.flush_events(); plt.pause(0.01)

    plt.ioff(); print("Closed.")

if __name__ == "__main__": main()

Loading models...
Models loaded.
Running... Trace will appear as faint dots.
Closed.


# center 오 이거 괜찮은데?

In [4]:
'''
[Local Player] Center Reward Viewer
- Features: Time Step Display, Trace Line, Real-time Control.
'''

import torch
import torch.nn as nn
import numpy as np
import math
import matplotlib
try: matplotlib.use('TkAgg')
except: pass
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# ==========================================
# 1. Definitions (Same as Server)
# ==========================================
class DoublePendulumEnv:
    def __init__(self):
        self.gravity = 9.8; self.masscart = 1.0
        self.masspole1 = 0.5; self.masspole2 = 0.5
        self.length1 = 0.5; self.length2 = 0.5
        self.force_mag = 40.0; self.dt = 0.02
        self.x_threshold = 4.0
        self.friction_cart = 5e-4; self.friction_pole = 2e-3
        self.reset()

    def reset(self):
        # Start from BOTTOM to verify swing-up
        th1 = np.pi; th2 = np.pi 
        self.physical_state = np.array([0, 0, th1, 0, th2, 0], dtype=np.float32)
        return self.get_obs()

    def get_obs(self):
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        return np.array([x, x_dot, np.sin(th1), np.cos(th1), th1_dot, np.sin(th2), np.cos(th2), th2_dot], dtype=np.float32)

    def step(self, action):
        raw_action = np.clip(action, -1.0, 1.0)[0]
        force = np.sign(raw_action) * (raw_action ** 2) * self.force_mag
        
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        s1, c1, s2, c2 = math.sin(th1), math.cos(th1), math.sin(th2), math.cos(th2)
        s12, c12 = math.sin(th1 - th2), math.cos(th1 - th2)
        m1, m2, M, l1, l2, g = self.masspole1, self.masspole2, self.masscart, self.length1, self.length2, self.gravity

        term1 = (m1 + m2) * l1 * th1_dot**2 * s1
        term2 = m2 * l2 * th2_dot**2 * s2
        f_c, f_p1, f_p2 = -self.friction_cart*x_dot, -self.friction_pole*th1_dot, -self.friction_pole*th2_dot
        eff_force = force + f_c
        
        torque1 = (M + m1 + m2) * g * s1 - term1 * c1 - term2 * c12 + f_p1
        th1_acc = (torque1 - eff_force * c1) / ((4/3) * (M + m1 + m2) * l1)
        th2_acc = (g * s2 - th1_acc * l1 * c12 + f_p2/l2) / (l2 * (4/3))
        x_acc = (eff_force - term1 - term2) / (M + m1 + m2)

        x_dot += x_acc * self.dt; x += x_dot * self.dt
        th1_dot += th1_acc * self.dt; th1 += th1_dot * self.dt
        th2_dot += th2_acc * self.dt; th2 += th2_dot * self.dt
        
        if x < -self.x_threshold: x = -self.x_threshold; x_dot = 0.0
        elif x > self.x_threshold: x = self.x_threshold; x_dot = 0.0
            
        self.physical_state = np.array([x, x_dot, th1, th1_dot, th2, th2_dot], dtype=np.float32)
        return self.get_obs()

class SparseAutoEncoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 128), nn.ELU(), nn.Linear(128, 64), nn.ELU(), nn.Linear(64, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 64), nn.ELU(), nn.Linear(64, 128), nn.ELU(), nn.Linear(128, input_dim))
    def forward(self, x): return None, self.encoder(x)

class PPOActor(nn.Module):
    def __init__(self, latent_dim=32, action_dim=1):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(latent_dim, 128), nn.Tanh(), nn.Linear(128, 64), nn.Tanh())
        self.mu = nn.Linear(64, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
    def forward(self, z): return torch.tanh(self.mu(self.net(z))), None

# ==========================================
# 2. Main Player
# ==========================================
def main():
    device = torch.device("cpu")
    print("Loading...")
    try:
        sae = SparseAutoEncoder(8, 32).to(device)
        actor = PPOActor(32, 1).to(device)
        sae.load_state_dict(torch.load("sae_center.pth", map_location=device))
        actor.load_state_dict(torch.load("actor_center.pth", map_location=device))
        sae.eval(); actor.eval()
        print("Models loaded.")
    except Exception as e: print(f"Error: {e}"); return

    env = DoublePendulumEnv()
    obs = env.reset()
    
    plt.ion()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(-5, 5); ax.set_ylim(-3, 3); ax.set_aspect('equal')
    ax.grid(True); ax.axvline(-4.0, c='k', ls='--'); ax.axvline(4.0, c='k', ls='--')
    
    # Trace Line
    trace_x, trace_y = [], []
    trace_line, = ax.plot([], [], 'o', color='gray', ms=1, alpha=0.3)
    
    cart, = ax.plot([], [], 'ks', ms=20, label='Cart')
    pole1, = ax.plot([], [], 'r-', lw=5, label='Inner')
    pole2, = ax.plot([], [], 'b-', lw=5, label='Outer')
    
    # [Add] Time Text
    info_text = ax.text(0.05, 0.9, '', transform=ax.transAxes, fontsize=12)
    ax.legend(loc='upper right')
    
    print("Running...")
    t_step = 0
    
    while plt.fignum_exists(fig.number):
        s_t = torch.from_numpy(obs).float().unsqueeze(0).to(device)
        with torch.no_grad(): _, z = sae(s_t); mu, _ = actor(z)
        action = mu.item()
        
        obs = env.step([action])
        x, _, th1, _, th2, _ = env.physical_state
        
        p1x = x + 2*env.length1*math.sin(th1); p1y = 2*env.length1*math.cos(th1)
        p2x = p1x + 2*env.length2*math.sin(th2); p2y = p1y + 2*env.length2*math.cos(th2)
        
        # Trace Update
        trace_x.append(p2x); trace_y.append(p2y)
        if len(trace_x) > 500: trace_x.pop(0); trace_y.pop(0)
        trace_line.set_data(trace_x, trace_y)
        
        cart.set_data([x], [0])
        pole1.set_data([x, p1x], [0, p1y])
        pole2.set_data([p1x, p2x], [p1y, p2y])
        
        # [Add] Text Update
        info_text.set_text(f"Time: {t_step}\nAction: {action:.2f}")
        t_step += 1
        
        fig.canvas.draw(); fig.canvas.flush_events(); plt.pause(0.01)

    plt.ioff(); print("Closed.")

if __name__ == "__main__": main()

Loading...
Models loaded.
Running...
Closed.


# 4가지 방법
1. 물속에서 실험
2. 저크 패널티
3. 시뮬레이션 주파수 증가
4. 이동거리 패널티

In [1]:
'''
[Local Player] GPU-Trained Curriculum Model Viewer
'''
import torch
import torch.nn as nn
import numpy as np
import math
import matplotlib
try: matplotlib.use('TkAgg')
except: pass
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

class DoublePendulumEnv:
    def __init__(self):
        self.masscart = 1.0; self.masspole1 = 0.5; self.masspole2 = 0.5
        self.length1 = 0.5; self.length2 = 0.5
        self.force_mag = 40.0; self.dt = 0.002 # 500Hz
        self.x_threshold = 4.0
        # Evaluation is always Real Physics
        self.gravity = 9.8
        self.friction_cart = 0.0005
        self.friction_pole = 0.002
        self.reset()

    def reset(self):
        th1 = np.pi; th2 = np.pi 
        self.physical_state = np.array([0, 0, th1, 0, th2, 0], dtype=np.float32)
        self.last_action = 0.0
        return self.get_obs()

    def get_obs(self):
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        return np.array([x, x_dot, np.sin(th1), np.cos(th1), th1_dot, np.sin(th2), np.cos(th2), th2_dot], dtype=np.float32)

    def step(self, action):
        raw_action = np.clip(action, -1.0, 1.0)[0]
        force = np.sign(raw_action) * (raw_action ** 2) * self.force_mag
        
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        s1, c1 = math.sin(th1), math.cos(th1)
        s2, c2 = math.sin(th2), math.cos(th2)
        s12, c12 = math.sin(th1 - th2), math.cos(th1 - th2)
        m1, m2, M, l1, l2, g = self.masspole1, self.masspole2, self.masscart, self.length1, self.length2, self.gravity

        term1 = (m1 + m2) * l1 * th1_dot**2 * s1
        term2 = m2 * l2 * th2_dot**2 * s2
        
        f_c = -self.friction_cart * x_dot
        f_p1 = -self.friction_pole * th1_dot
        f_p2 = -self.friction_pole * th2_dot
        
        eff_force = force + f_c
        
        torque1 = (M + m1 + m2) * g * s1 - term1 * c1 - term2 * c12 + f_p1
        th1_acc = (torque1 - eff_force * c1) / ((4/3) * (M + m1 + m2) * l1)
        th2_acc = (g * s2 - th1_acc * l1 * c12 + f_p2/l2) / (l2 * (4/3))
        x_acc = (eff_force - term1 - term2) / (M + m1 + m2)

        x_dot += x_acc * self.dt; x += x_dot * self.dt
        th1_dot += th1_acc * self.dt; th1 += th1_dot * self.dt
        th2_dot += th2_acc * self.dt; th2 += th2_dot * self.dt
        
        if x < -self.x_threshold: x = -self.x_threshold; x_dot = 0.0
        elif x > self.x_threshold: x = self.x_threshold; x_dot = 0.0
            
        self.physical_state = np.array([x, x_dot, th1, th1_dot, th2, th2_dot], dtype=np.float32)
        return self.get_obs()

class SparseAutoEncoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 512), nn.ELU(), nn.Linear(512, 256), nn.ELU(), nn.Linear(256, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 256), nn.ELU(), nn.Linear(256, 512), nn.ELU(), nn.Linear(512, input_dim))
    def forward(self, x): return None, self.encoder(x)

class PPOActor(nn.Module):
    def __init__(self, latent_dim=64, action_dim=1):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(latent_dim, 512), nn.Tanh(), nn.Linear(512, 256), nn.Tanh())
        self.mu = nn.Linear(256, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
    def forward(self, z): return torch.tanh(self.mu(self.net(z))), None

def main():
    device = torch.device("cpu")
    print("Loading models...")
    try:
        sae = SparseAutoEncoder(8, 64).to(device)
        actor = PPOActor(64, 1).to(device)
        sae.load_state_dict(torch.load("sae_gpu_final.pth", map_location=device))
        actor.load_state_dict(torch.load("actor_gpu_final.pth", map_location=device))
        sae.eval(); actor.eval()
        print("Models loaded.")
    except Exception as e: print(f"Error: {e}"); return

    env = DoublePendulumEnv()
    obs = env.reset()
    
    plt.ion()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(-5, 5); ax.set_ylim(-3, 3); ax.set_aspect('equal'); ax.grid(True)
    ax.axvline(-4, c='k', ls='--'); ax.axvline(4, c='k', ls='--')
    cart, = ax.plot([], [], 'ks', ms=20)
    pole1, = ax.plot([], [], 'r-', lw=5); pole2, = ax.plot([], [], 'b-', lw=5)
    txt = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    print("Running...")
    while plt.fignum_exists(fig.number):
        for _ in range(10): # Speed up visualization
            s_t = torch.from_numpy(obs).float().unsqueeze(0).to(device)
            with torch.no_grad(): _, z = sae(s_t); mu, _ = actor(z)
            obs = env.step([mu.item()])
            
        x, _, th1, _, th2, _ = env.physical_state
        p1x = x + 2*0.5*math.sin(th1); p1y = 2*0.5*math.cos(th1)
        p2x = p1x + 2*0.5*math.sin(th2); p2y = p1y + 2*0.5*math.cos(th2)
        
        cart.set_data([x], [0])
        pole1.set_data([x, p1x], [0, p1y]); pole2.set_data([p1x, p2x], [p1y, p2y])
        txt.set_text(f"Action: {mu.item():.2f}")
        fig.canvas.draw(); fig.canvas.flush_events()

if __name__ == "__main__": main()

Loading models...
Models loaded.
Running...


## 균형잡기

In [3]:
'''
[Local Player] Precision Visualizer
- Feature: Real-time Reward Bar (Shows how well it's balancing)
'''
import torch
import torch.nn as nn
import numpy as np
import math
import matplotlib
try: matplotlib.use('TkAgg')
except: pass
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Env & Models (Same as Server)
class PrecisionEnv:
    def __init__(self):
        self.masscart = 1.0; self.masspole1 = 0.5; self.masspole2 = 0.5
        self.length1 = 0.5; self.length2 = 0.5; self.force_mag = 40.0; self.dt = 0.02
        self.x_threshold = 4.0; self.friction_cart = 5e-4; self.friction_pole = 2e-3
        self.reset()
    def reset(self):
        th1 = np.pi; th2 = np.pi
        self.physical_state = np.array([0, 0, th1, 0, th2, 0], dtype=np.float32)
        return self.get_obs()
    def get_obs(self):
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        return np.array([x, x_dot, np.sin(th1), np.cos(th1), th1_dot, np.sin(th2), np.cos(th2), th2_dot], dtype=np.float32)
    def step(self, action):
        raw_action = np.clip(action, -1.0, 1.0)[0]
        force = np.sign(raw_action) * (raw_action ** 2) * self.force_mag
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        s1, c1, s2, c2 = math.sin(th1), math.cos(th1), math.sin(th2), math.cos(th2)
        s12, c12 = math.sin(th1 - th2), math.cos(th1 - th2)
        m1, m2, M, l1, l2, g = 0.5, 0.5, 1.0, 0.5, 0.5, 9.8
        term1 = (m1+m2)*l1*th1_dot**2*s1; term2 = m2*l2*th2_dot**2*s2
        f_c, f_p1, f_p2 = -self.friction_cart*x_dot, -self.friction_pole*th1_dot, -self.friction_pole*th2_dot
        eff_force = force + f_c
        torque1 = (M+m1+m2)*g*s1 - term1*c1 - term2*c12 + f_p1
        th1_acc = (torque1 - eff_force*c1)/((4/3)*(M+m1+m2)*l1)
        th2_acc = (g*s2 - th1_acc*l1*c12 + f_p2/l2)/(l2*(4/3))
        x_acc = (eff_force - term1 - term2)/(M+m1+m2)
        x_dot += x_acc*self.dt; x += x_dot*self.dt
        th1_dot += th1_acc*self.dt; th1 += th1_dot*self.dt
        th2_dot += th2_acc*self.dt; th2 += th2_dot*self.dt
        
        # Calculate Reward for Visualization
        th1_n = (th1 + np.pi) % (2 * np.pi) - np.pi
        th2_n = (th2 + np.pi) % (2 * np.pi) - np.pi
        r_angle = np.exp(-1.0*th1_n**2) * np.exp(-2.0*th2_n**2)
        r_vel = np.exp(-0.1*th1_dot**2) * np.exp(-0.1*th2_dot**2)
        current_quality = r_angle * r_vel
        
        if x<-4:
            x=-4; x_dot=0
        
        elif x>4:
            x=4; x_dot=0
        self.physical_state = np.array([x, x_dot, th1, th1_dot, th2, th2_dot], dtype=np.float32)
        return self.get_obs(), current_quality

class SparseAutoEncoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 128), nn.ELU(), nn.Linear(128, 64), nn.ELU(), nn.Linear(64, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 64), nn.ELU(), nn.Linear(64, 128), nn.ELU(), nn.Linear(128, input_dim))
    def forward(self, x): return None, self.encoder(x)

class PPOActor(nn.Module):
    def __init__(self, latent_dim=32, action_dim=1):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(latent_dim, 128), nn.Tanh(), nn.Linear(128, 64), nn.Tanh())
        self.mu = nn.Linear(64, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
    def forward(self, z): return torch.tanh(self.mu(self.net(z))), None

def main():
    device = torch.device("cpu")
    print("Loading...")
    try:
        sae = SparseAutoEncoder(8, 32).to(device)
        actor = PPOActor(32, 1).to(device)
        sae.load_state_dict(torch.load("sae_precision.pth", map_location=device))
        actor.load_state_dict(torch.load("actor_precision.pth", map_location=device))
        sae.eval(); actor.eval()
    except: print("Files Missing."); return

    env = PrecisionEnv()
    obs = env.reset()
    
    plt.ion()
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), gridspec_kw={'height_ratios': [3, 1]})
    
    # 1. Animation
    ax1.set_xlim(-5, 5); ax1.set_ylim(-3, 3); ax1.set_aspect('equal'); ax1.grid(True)
    ax1.axvline(-4, c='k', ls='--'); ax1.axvline(4, c='k', ls='--')
    cart, = ax1.plot([], [], 'ks', ms=20)
    pole1, = ax1.plot([], [], 'r-', lw=5)
    pole2, = ax1.plot([], [], 'b-', lw=5)
    txt = ax1.text(0.05, 0.9, '', transform=ax1.transAxes)
    
    # 2. Reward Bar
    ax2.set_xlim(0, 1); ax2.set_ylim(0, 1)
    ax2.set_title("Stability Score (Angle * Velocity)")
    bar = ax2.barh([0.5], [0], height=0.3, color='green')
    
    print("Running...")
    while plt.fignum_exists(fig.number):
        s_t = torch.from_numpy(obs).float().unsqueeze(0).to(device)
        with torch.no_grad(): _, z = sae(s_t); mu, _ = actor(z)
        
        obs, quality = env.step([mu.item()])
        x, _, th1, _, th2, _ = env.physical_state
        
        # Render CartPole
        p1x = x + 2*0.5*math.sin(th1); p1y = 2*0.5*math.cos(th1)
        p2x = p1x + 2*0.5*math.sin(th2); p2y = p1y + 2*0.5*math.cos(th2)
        cart.set_data([x], [0])
        pole1.set_data([x, p1x], [0, p1y])
        pole2.set_data([p1x, p2x], [p1y, p2y])
        txt.set_text(f"Action: {mu.item():.2f}")
        
        # Render Bar
        bar[0].set_width(quality)
        bar[0].set_color('red' if quality < 0.5 else 'green')
        
        fig.canvas.draw(); fig.canvas.flush_events()

if __name__ == "__main__": main()

Loading...
Running...


# 논문 참고

In [3]:
'''
[Local Player] Paper Reward Viewer
'''
import torch
import torch.nn as nn
import numpy as np
import math
import matplotlib
try: matplotlib.use('TkAgg')
except: pass
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Env Definitions (Must Match)
class DoublePendulumEnv:
    def __init__(self):
        self.masscart = 1.0; self.masspole1 = 0.5; self.masspole2 = 0.5
        self.length1 = 0.5; self.length2 = 0.5
        self.force_mag = 40.0; self.dt = 0.02
        self.x_threshold = 2.4 # Paper spec
        self.friction_cart = 0.05
        self.friction_pole = 0.001
        self.reset()

    def reset(self):
        th1 = np.pi; th2 = np.pi 
        self.physical_state = np.array([0, 0, th1, 0, th2, 0], dtype=np.float32)
        return self.get_obs()

    def get_obs(self):
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        return np.array([x, x_dot, np.sin(th1), np.cos(th1), th1_dot, np.sin(th2), np.cos(th2), th2_dot], dtype=np.float32)

    def step(self, action):
        raw_action = np.clip(action, -1.0, 1.0)[0]
        force = np.sign(raw_action) * (raw_action ** 2) * self.force_mag
        x, x_dot, th1, th1_dot, th2, th2_dot = self.physical_state
        s1, c1, s2, c2 = math.sin(th1), math.cos(th1), math.sin(th2), math.cos(th2)
        s12, c12 = math.sin(th1 - th2), math.cos(th1 - th2)
        m1, m2, M, l1, l2, g = 0.5, 0.5, 1.0, 0.5, 0.5, 9.8
        term1 = (m1 + m2) * l1 * th1_dot**2 * s1
        term2 = m2 * l2 * th2_dot**2 * s2
        
        f_c = -self.friction_cart * x_dot
        f_p1 = -self.friction_pole * th1_dot
        f_p2 = -self.friction_pole * th2_dot
        eff_force = force + f_c
        
        torque1 = (M + m1 + m2) * g * s1 - term1 * c1 - term2 * c12 + f_p1
        th1_acc = (torque1 - eff_force * c1) / ((4/3) * (M + m1 + m2) * l1)
        th2_acc = (g * s2 - th1_acc * l1 * c12 + f_p2/l2) / (l2 * (4/3))
        x_acc = (eff_force - term1 - term2) / (M + m1 + m2)

        x_dot += x_acc * self.dt; x += x_dot * self.dt
        th1_dot += th1_acc * self.dt; th1 += th1_dot * self.dt
        th2_dot += th2_acc * self.dt; th2 += th2_dot * self.dt
        
        th1 = (th1 + np.pi) % (2 * np.pi) - np.pi
        th2 = (th2 + np.pi) % (2 * np.pi) - np.pi
        
        if x < -self.x_threshold: x = -self.x_threshold; x_dot = 0
        elif x > self.x_threshold: x = self.x_threshold; x_dot = 0
            
        self.physical_state = np.array([x, x_dot, th1, th1_dot, th2, th2_dot], dtype=np.float32)
        return self.get_obs()

class SparseAutoEncoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 512), nn.ELU(), nn.Linear(512, 256), nn.ELU(), nn.Linear(256, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 256), nn.ELU(), nn.Linear(256, 512), nn.ELU(), nn.Linear(512, input_dim))
    def forward(self, x): return None, self.encoder(x)

class PPOActor(nn.Module):
    def __init__(self, latent_dim=64, action_dim=1):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(latent_dim, 512), nn.Tanh(), nn.Linear(512, 256), nn.Tanh())
        self.mu = nn.Linear(256, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
    def forward(self, z): return torch.tanh(self.mu(self.net(z))), None

def main():
    device = torch.device("cpu")
    try:
        sae = SparseAutoEncoder(8, 64).to(device)
        actor = PPOActor(64, 1).to(device)
        sae.load_state_dict(torch.load("sae_paper.pth", map_location=device))
        actor.load_state_dict(torch.load("actor_paper.pth", map_location=device))
        sae.eval(); actor.eval()
        print("Models loaded.")
    except: print("Files missing."); return

    env = DoublePendulumEnv()
    obs = env.reset()
    
    plt.ion()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(-5, 5); ax.set_ylim(-3, 3); ax.set_aspect('equal'); ax.grid(True)
    ax.axvline(-2.4, c='k', ls='--'); ax.axvline(2.4, c='k', ls='--')
    cart, = ax.plot([], [], 'ks', ms=20)
    pole1, = ax.plot([], [], 'r-', lw=5); pole2, = ax.plot([], [], 'b-', lw=5)
    txt = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    while plt.fignum_exists(fig.number):
        for _ in range(10):
            s_t = torch.from_numpy(obs).float().unsqueeze(0).to(device)
            with torch.no_grad(): _, z = sae(s_t); mu, _ = actor(z)
            obs = env.step([mu.item()])
            
        x, _, th1, _, th2, _ = env.physical_state
        p1x = x + 2*0.5*math.sin(th1); p1y = 2*0.5*math.cos(th1)
        p2x = p1x + 2*0.5*math.sin(th2); p2y = p1y + 2*0.5*math.cos(th2)
        
        cart.set_data([x], [0])
        pole1.set_data([x, p1x], [0, p1y]); pole2.set_data([p1x, p2x], [p1y, p2y])
        txt.set_text(f"Action: {mu.item():.2f}")
        fig.canvas.draw(); fig.canvas.flush_events()

if __name__ == "__main__": main()

AttributeError: partially initialized module 'torch' has no attribute 'fx' (most likely due to a circular import)